In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

In [ ]:
[deps]
Colors = "5ae59095-9a9b-59fe-a467-6f913c188581"
Convex = "f65535da-76fb-5f13-bab9-19810c17039a"
ECOS = "e2685f51-7e38-5353-a97d-a921fd2c8199"
FiniteDiff = "6a86dc24-6348-571c-b903-95158fe2bd41"
ForwardDiff = "f6369f11-7733-5829-9624-2563aa707210"
Interpolations = "a98d9a8b-a2ab-59e6-89dd-64a1c18fca59"
Ipopt = "b6b21f68-93f8-5de0-b562-5493be1d77c9"
JLD2 = "033835bb-8acc-5ee8-8aae-3f567f8a3819"
MathOptInterface = "b8f27783-ece8-5eb3-8dc8-9495eed66fee"
MeshCat = "283c5d60-a78f-5afe-a0af-af636b173e11"
Plots = "91a5bcdd-55d7-5caf-9e0b-520d859cae80"
StaticArrays = "90137ffa-7385-5640-81b9-e52037218182"


In [5]:
import Pkg
Pkg.resolve()

Pkg.activate(@__DIR__)
Pkg.instantiate()
Pkg.add("Interpolations")
Pkg.add("Dierckx")
import Interpolations
import Dierckx
import FiniteDiff
import ForwardDiff as FD
import Convex as cvx
import ECOS
using Dierckx
using Interpolations
using LinearAlgebra
using Plots
using Random
using JLD2
using Test
using MeshCat
const mc = MeshCat
using StaticArrays
using Printf

  No Changes to `~/16745/final_proj/Project.toml`
  No Changes to `~/16745/final_proj/Manifest.toml`
  Activating environment at `~/16745/final_proj/Project.toml`
    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
  No Changes to `~/16745/final_proj/Project.toml`
  No Changes to `~/16745/final_proj/Manifest.toml`
   Resolving package versions...
  No Changes to `~/16745/final_proj/Project.toml`
  No Changes to `~/16745/final_proj/Manifest.toml`


In [ ]:
versioninfo()

In [2]:
# from HW 3

import MathOptInterface as MOI
import Ipopt
import FiniteDiff
import ForwardDiff
using LinearAlgebra

struct ProblemMOI <: MOI.AbstractNLPEvaluator
    n_nlp::Int
    m_nlp::Int
    obj_grad::Bool
    con_jac::Bool
    sparsity_jac
    sparsity_hess
    hessian_lagrangian::Bool
    params::NamedTuple
    cost # ::Function
    con # ::Function
    diff_type # :Symbol
end

function ProblemMOI(n_nlp,m_nlp,params,cost,con,diff_type;
        obj_grad=true,
        con_jac=true,
        sparsity_jac=sparsity_jacobian(n_nlp,m_nlp),
        sparsity_hess=sparsity_hessian(n_nlp,m_nlp),
        hessian_lagrangian=false)

    ProblemMOI(n_nlp,m_nlp,
        obj_grad,
        con_jac,
        sparsity_jac,
        sparsity_hess,
        hessian_lagrangian,
        params,
        cost,
        con,
        diff_type)
end

function row_col!(row,col,r,c)
    for cc in c
        for rr in r
            push!(row,convert(Int,rr))
            push!(col,convert(Int,cc))
        end
    end
    return row, col
end

function sparsity_jacobian(n,m)

    row = []
    col = []

    r = 1:m
    c = 1:n

    row_col!(row,col,r,c)

    return collect(zip(row,col))
end

function sparsity_hessian(n,m)

    row = []
    col = []

    r = 1:m
    c = 1:n

    row_col!(row,col,r,c)

    return collect(zip(row,col))
end

function MOI.eval_objective(prob::MOI.AbstractNLPEvaluator, x)
    prob.cost(prob.params, x)
end

function MOI.eval_objective_gradient(prob::MOI.AbstractNLPEvaluator, grad_f, x)
    _cost(_x) = prob.cost(prob.params, _x)
    if prob.diff_type == :auto
        ForwardDiff.gradient!(grad_f,_cost,x)
    else
        FiniteDiff.finite_difference_gradient!(grad_f, _cost, x)
    end
    return nothing
end

function MOI.eval_constraint(prob::MOI.AbstractNLPEvaluator,c,x)
    c .= prob.con(prob.params, x)
    return nothing
end

function MOI.eval_constraint_jacobian(prob::MOI.AbstractNLPEvaluator, jac, x)
    _con(_x) = prob.con(prob.params, _x)
    if prob.diff_type == :auto
        reshape(jac,prob.m_nlp,prob.n_nlp) .= ForwardDiff.jacobian(_con, x)
    else
        reshape(jac,prob.m_nlp,prob.n_nlp) .= FiniteDiff.finite_difference_jacobian(_con, x)
    end
    return nothing
end

function MOI.features_available(prob::MOI.AbstractNLPEvaluator)
    return [:Grad, :Jac]
end

MOI.initialize(prob::MOI.AbstractNLPEvaluator, features) = nothing
MOI.jacobian_structure(prob::MOI.AbstractNLPEvaluator) = prob.sparsity_jac


"""
x = fmincon(cost,equality_constraint,inequality_constraint,x_l,x_u,c_l,c_u,x0,params,diff_type)

This function uses IPOPT to minimize an objective function

`cost(params, x)`

With the following three constraints:

`equality_constraint(params, x) = 0`
`c_l <= inequality_constraint(params, x) <= c_u`
`x_l <= x <= x_u`

Problem specific parameters should be loaded into params::NamedTuple (things like
cost weights, dynamics parameters, etc.).

args:
    cost::Function                    - objective function to be minimzed (returns scalar)
    equality_constraint::Function     - c_eq(params, x) == 0
    inequality_constraint::Function   - c_l <= c_ineq(params, x) c_u
    x_l::Vector                       - x_l <= x <= x_u
    x_u::Vector                       - x_l <= x <= x_u
    c_l::Vector                       - c_l <= c_ineq(params, x) <= x_u
    c_u::Vector                       - c_l <= c_ineq(params, x) <= x_u
    x0::Vector                        - initial guess
    params::NamedTuple                - problem parameters for use in costs/constraints
    diff_type::Symbol                 - :auto for ForwardDiff, :finite for FiniteDiff

optional args:
    tol                               - optimality tolerance
    c_tol                             - constraint violation tolerance
    max_iters                         - max iterations
    verbose                           - true for IPOPT output, false for nothing

You should try and use :auto for your `diff_type` first, and only use :finite if you
absolutely cannot get ForwardDiff to work.

This function will run a few basic checks before sending the problem off to IPOPT to
solve. The outputs of these checks will be reported as the following:

---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :auto (ForwardDiff.jl)----
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------

If you're getting stuck during the testing of one of the derivatives, try switching
to FiniteDiff.jl by setting diff_type = :finite.
"""

function fmincon(cost::Function,
                 equality_constraint::Function,
                 inequality_constraint::Function,
                 x_l::Vector,
                 x_u::Vector,
                 c_l::Vector,
                 c_u::Vector,
                 x0::Vector,
                 params::NamedTuple,
                 diff_type::Symbol;
                 tol = 1e-4,
                 c_tol = 1e-4,
                 max_iters = 1_000,
                 verbose = true)::Vector

    n_primals = length(x0)
    n_eq = length(equality_constraint(params, x0))
    n_ineq = length(inequality_constraint(params, x0))

    verbose && println("---------checking dimensions of everything----------")
    @assert length(x0) == length(x_l) == length(x_u)
    @assert length(c_l) == length(c_u) == n_ineq
    @assert all(x_u .>= x_l)
    if n_ineq > 0
        @assert all(c_u .>= c_l)
    end
    verbose && println("---------all dimensions good------------------------")


    function con(params, x)
        [
            equality_constraint(params, x);
            inequality_constraint(params, x)
        ]
    end

    if diff_type == :auto
        verbose && println("---------diff type set to :auto (ForwardDiff.jl)----")
    else
        verbose && println("---------diff type set to :finite (FiniteDiff.jl)---")
    end
    verbose && println("---------testing objective gradient-----------------")
    if diff_type == :auto
        ForwardDiff.gradient(_x -> cost(params, _x), x0)
    else
        FiniteDiff.finite_difference_gradient(_x -> cost(params, _x), x0)
    end
    verbose && println("---------testing constraint Jacobian----------------")
    if diff_type == :auto
        ForwardDiff.jacobian(_x -> con(params, _x), x0)
    else
        FiniteDiff.finite_difference_jacobian(_x -> con(params, _x), x0)
    end
    verbose && println("---------successfully compiled both derivatives-----")

    prob = ProblemMOI(n_primals, n_eq + n_ineq, params, cost, con, diff_type)

    # add zeros(n_eq) for equality constraint
    nlp_bounds = MOI.NLPBoundsPair.([zeros(n_eq); c_l],[zeros(n_eq); c_u])
    block_data = MOI.NLPBlockData(nlp_bounds, prob, true)

    solver = Ipopt.Optimizer()
    solver.options["max_iter"] = max_iters
    solver.options["tol"] = tol
    solver.options["constr_viol_tol"] = c_tol

    if verbose
        solver.options["print_level"] = 5
    else
        solver.options["print_level"] = 0
    end

    x = MOI.add_variables(solver,prob.n_nlp)

    for i = 1:prob.n_nlp
        MOI.add_constraint(solver, x[i], MOI.LessThan(x_u[i]))
        MOI.add_constraint(solver, x[i], MOI.GreaterThan(x_l[i]))
        MOI.set(solver, MOI.VariablePrimalStart(), x[i], x0[i])
    end

    # Solve the problem
    verbose && println("---------IPOPT beginning solve----------------------")

    MOI.set(solver, MOI.NLPBlock(), block_data)
    MOI.set(solver, MOI.ObjectiveSense(), MOI.MIN_SENSE)
    MOI.optimize!(solver)

    # Get the solution
    res = MOI.get(solver, MOI.VariablePrimal(), x)

    return res

end



fmincon (generic function with 1 method)

In [113]:
function dynamics(model::NamedTuple, x::Vector, u::Vector)::Vector
    # kinematic bicycle model continuous time dynamics
    # lr = distance from rear axle to COM
    # L = wheelbase

    lr, L = model.lr, model.L
    # reftraj, reftraj_deriv, reftraj_2nd_deriv =
    #     model.reftraj,
    #     model.reftraj_deriv,
    #     model.reftraj_2nd_deriv

    # reftraj(s) = x, y
    # a = a(s), which is experimentally determined

    # along track, cross track, heading w.r.t track, steering angle, speed
    s, r, e_psi, δ, v = x

    # rate of steer
    δdot, a = u
    βdot = δdot / (1 + (lr * δ / L)^2)

    sin_e_psi, cos_e_psi = sincos(e_psi)

    # function T(_s)
    #     reftrajdot = derivative(reftraj, _s)
    #     reftrajdot / norm(reftrajdot)
    # end

    Tdot = reftraj_2nd_deriv(get_idx_from_dist(s))[1] / norm(reftraj_deriv(get_idx_from_dist(s))[1])

    # curvature of reftraj
    κ = norm(Tdot)

    sdot = v * cos_e_psi / (1 - r * κ)
    rdot = v * sin_e_psi
    e_psi_dot = βdot + sdot * κ

    xdot = [
        sdot,
        rdot,
        e_psi_dot,
        δdot,
        a
    ]
    print(xdot)
    return xdot
end

function rk4(model::NamedTuple, ode::Function, x::Vector, u::Vector, dt::Real)::Vector
    k1 = dt * ode(model, x,        u)
    k2 = dt * ode(model, x + k1/2, u)
    k3 = dt * ode(model, x + k2/2, u)
    k4 = dt * ode(model, x + k3,   u)
    return x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end

rk4 (generic function with 1 method)

In [120]:
function generate_reftraj(_X)::ParametricSpline
    # https://juliahub.com/ui/Packages/General/Dierckx
    xs = 1.0:size(_X, 1)
    itp = ParametricSpline(xs, _X'; k=3, bc="extrapolate")
    return itp
end

X_mat = [0 0;0 1.0;0 2;0 4;0 9;0 16;0 25]
X = [X_mat[i,:] for i in 1:size(X_mat,1)]
reftraj_spl = generate_reftraj(X_mat)

function matrix_to_vec(mat)
    vec = [mat[i,:] for i in 1:size(mat,1)]
    return vec
end

function reftraj(i)
    mat = transpose(reftraj_spl(i))
    return matrix_to_vec(mat)
end

function reftraj_deriv(i)
    mat = transpose(derivative(reftraj_spl, i))
    return matrix_to_vec(mat)
end

function reftraj_2nd_deriv(i)
    mat = transpose(derivative(reftraj_spl, i; nu=2))
    return matrix_to_vec(mat)
end

function generate_distances()
    dt = 0.0001
    ts = 0.0:dt:size(X, 1)
    drdts = copy(reftraj_deriv(ts))
    print("drdts shape", size(drdts))
    drdts = [drdts[i, :] for i in 1:size(drdts, 1)]
    speeds = norm.(eachrow(drdts))

    distances = zeros(size(speeds, 1))
    distances[1] = speeds[1] * dt
    for i = 2:size(drdts, 1)
        distances[i] = speeds[i] * dt + distances[i - 1]
    end

    return distances, ts
end
distances, ts = generate_distances()
print("size of dist", size(distances), "size of ts", size(ts))

# 1-indexed, so add 1 since ts are 0-indexed
get_idx_from_dist(_d) =
    1 + linear_interpolation(
    distances,
    ts,
    extrapolation_bc=Interpolations.Line())(_d)

get_dist_from_idx(_i) =
    linear_interpolation(
    ts,
    distances,
    extrapolation_bc=Interpolations.Line())(_i - 1)

get_pos_from_dist(_d) =
    reftraj(get_idx_from_dist(_d))

function get_dist_from_pos(x)
    # compute distance to each knot pt
    dists_to_path = norm.(X .- Ref(x))
    knot_idx = argmin(dists_to_path)
    # subsample the distance function in the interval [knot idx - 1, knot idx + 1]
    # for each subsample along the path, compute distance from x  to subsample
    path_subsample = reftraj(knot_idx-1:0.01:knot_idx+1)
    dists_to_subsample = norm.(path_subsample .- Ref(x))
    knot_idx_subsample = argmin(dists_to_subsample) * 0.01 + (knot_idx - 1)
    @show "subsample range", knot_idx-1:0.01:knot_idx+1
    @show "spline position along subsample range", path_subsample
    @show "subsampled dists" dists_to_subsample
    @show "closest idx", knot_idx
    @show "closest subsample idx", knot_idx_subsample
    return get_dist_from_idx(knot_idx_subsample)
end

get_dist_from_pos (generic function with 1 method)

In [119]:
# adapted from HW4
function create_idx(nx,nu,N)
    # create idx for indexing convenience
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    # and stacked dynamics constraints of size nx are
    # c[idx.c[i]] = <dynamics constraint at time step i>

    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]

    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx

    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end

function dynamics_constraints(params, Z)
    idx = params.idx
    N = params.N
    c = zeros(eltype(Z), idx.nc)
    model = params.model
    dt = params.dt

    for i = 1:N-1
        xi = Z[idx.x[i]]
        xi1 = Z[idx.x[i + 1]]
        ui = Z[idx.u[i]]
        c[idx.c[i]] = rk4(model, dynamics, xi, ui, dt) - xi1
    end

    return c
end

function equality_constraints(params, Z)
    idx, xi = params.idx, params.xi
    # TODO: stack up all of our equality constraints
    # print("xi shape, x shape ", size(xi), size(Z[idx.x[1]]))
    init_constraint = xi - Z[idx.x[1]]
    return init_constraint
end

function inequality_constraints(params, Z)
    return [0]
end

function cost(params::NamedTuple, Z::Vector)::Real
    # cost function
    idx, N, s_goal = params.idx, params.N, params.s_goal
    Q, R = params.Q, params.R

    J = 0
    for i = 1:N-1
        Xe = Z[idx.x[i]] - [s_goal, 0, 0, 0, 0]
        U = Z[idx.u[i]]
        J += Xe' * Q * Xe + U' * R * U
    end

    Xfe = Z[idx.x[N]] - [s_goal, 0, 0, 0, 0]
    J += Xfe' * Q * Xfe

    return 0.5 * J
end

function optimize(Xpath)
    dt = 0.001 #s
    t_horizon = 2 #s
    N = Int(t_horizon / dt)

    model = (L = 1.2,
        lr = 0.4,
        reftraj_2nd_deriv=reftraj_2nd_deriv,
        reftraj_deriv=reftraj_deriv,
        reftraj=reftraj)

    track = (width = 3.0)
    idx = create_idx(5, 2, N)

    Q = diagm([100; 0; 0; 0; 0])
    R = diagm([0.1; 00.1])
    # print("len x, reftraj at len x ", length(Xpath), reftraj(length(Xpath)))
    s_goal = get_dist_from_idx(length(Xpath))
    # print("sgoal", s_goal)

    @show "Xpath", Xpath[1]
    si = get_dist_from_pos(Xpath[1])


    print(get_pos_from_dist(si), Xpath[1])
    # get pos from dist returns N x 2
    # N is 1 here so still need to index into the return value
    pos_to_ref = get_pos_from_dist(si)[1] - Xpath[1]

    # check sign via cross product with tangent
    tangent = reftraj_deriv(1)[1]
    @show size(tangent)
    cross_product = (tangent[1] * pos_to_ref[2]) - (tangent[2] * pos_to_ref[1])
    r_norm = norm(tangent[1] - pos_to_ref[2])
    # TODO: figure out sign convention
    ri = sign(cross_product) * r_norm
    xi = [si, ri, 0, 0, 0]
    @show ("xi: ", xi)

    params = (
        model=model,
        track=track,
        idx=idx,
        N=N,
        Q=Q,
        R=R,
        dt=dt,
        xi=xi,
        s_goal=s_goal)
    z0 = zeros(idx.nz)
    # TODO: if this isn't good enough, do constant velocity and get states from reftraj
    # and controls from curvature

    # adding a little noise to the initial guess is a good idea
    # z0 = z0 + (1e-6)*randn(idx.nz)

    # TODO: primal bounds

    x_l = -Inf*ones(idx.nz) # update this
    x_u =  Inf*ones(idx.nz) # update this

    for i = 1:N-1
        # cross track
        x_l[idx.x[i+1][2]] = -3.0
        x_u[idx.x[i+1][2]] = 3.0

        # accelerate/brake limits
        x_l[idx.u[i][1]] = -3.0
        x_u[idx.u[i][1]] = 3.0

        # steer rate limits
        x_l[idx.u[i][2]] = -1.0
        x_u[idx.u[i][2]] = 1.0
    end

    # inequality constraint bounds
    c_l =  -Inf*ones(1)
    c_u =   Inf*ones(1)
    diff_type = :auto

    Z = fmincon(cost,equality_constraints,inequality_constraints,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-6, c_tol = 1e-6, max_iters = 10_000, verbose = true)

    xi = [17.5, 0, 0, 0, 0]
    ui = [0.0, 0]
    print(rk4(model, dynamics, xi, ui, dt))

    for i = 1:N-1
        print(i, "state:", Z[idx.x[i]])
        print("ctrl:", Z[idx.u[i]])
        print("\n")
    end
    print("final state", Z[idx.x[N]])
    print("\n")
end

optimize(X)